# Machine Learning
기본 Processer

## 1 Structure
Input Data -> Inference(가설모델을 계산) -> Loss(손실계산) -> train(파라미터 갱신) -> Evaluation(평가)

In [1]:
import tensorflow as tf
def inference(x):pass
def loss(x,y):pass
def inputs() :pass 
def train(total_loss):pass
def evaluation(sexx, x, y):pass

In [2]:
with tf.Session() as sess: pass
#     tf.global_variables_initializer()
#     #x,y = inputs()
#     total_loss = loss(x,y)
#     train_operation = train(total_loss)
#     coord = tf.train.Coordinator()
#     threads = tf.train.start_queue_runners(sess = sess, coord = coord)
#     training_steps = 1000

#     for step in range(training_steps):
#         sess.run([train_operation])
#         if step % 10 == 0:  
#             print('loss :', sess.run(total_loss))

#     evaluation(sess, x ,y)
#     coord.request_stop()
#     coord.join(threads)
#     sess.close()

## 2 학습 Check Points
tf.train.Saver() : 이진파일로 학습Graph를 저장

In [3]:
# 학습 전 저장할 객체를 생성
#saver = tf.train.Saver()

# 학습 반복 중 모델을 저장 : -{step} 으로  my-mode-2000 체크포인트별 객체를 생성
# if step % 1000 == 0:
#     saver.save(sess, 'my-model-{step}', global_step = step )

# 학습이 끝난 모델객체를 저장
# saver.save(sess, 'my-model', global_step = training_steps)
# sess.close()

In [4]:
# tf.train.get_checkpoint_state() - 특정 지점에서 학습을 재개한다

# with tf.Session() as sess:
#     initial_step = 0
#     ckpt = tf.train.get_checkpoint_state(os.path.dirname(__file__))
#     if ckpt and ckpt.model_checkpoint_path:
#         saver.restore(sess, ckpt.model_checkpoint_path)
#         initial_step = int(ckpt.model_checkpoint_path.rsplit('-', 1)[1])

# #actual training loop
# for step in range(initial_step, training_steps):

## 3 Linear_Regression
선형회귀 : Continuous 적인 real number 를 예측시

In [5]:
import tensorflow as tf
# 가설함수 모델 만들기
W = tf.Variable(tf.zeros([2, 1]), name="weights")
b = tf.Variable(0., name="bias")

def inference(X):
    return tf.matmul(X, W) + b

In [6]:
# Sequare Error 를 활용한 가설함수 검증하기
def loss(X, Y):
    Y_predicted = inference(X)
    return tf.reduce_sum(tf.squared_difference(Y, Y_predicted))

In [7]:
# 입력데이터
# Data from http://people.sc.fsu.edu/~jburkardt/datasets/regression/x09.txt
def inputs():
    weight_age = [[84, 46], [73, 20], [65, 52], [70, 30], [76, 57], [69, 25], [63, 28],
                  [72, 36], [79, 57], [75, 44], [27, 24], [89, 31], [65, 52], [57, 23],
                  [59, 60], [69, 48], [60, 34], [79, 51], [75, 50], [82, 34], [59, 46],
                  [67, 23], [85, 37], [55, 40], [63, 30]]
    blood_fat_content = [354, 190, 405, 263, 451, 302, 288, 385, 402, 365, 209, 290, 346, 
                         254, 395, 434, 220, 374, 308, 220, 311, 181, 274, 303, 244]
    return tf.to_float(weight_age), tf.to_float(blood_fat_content)

In [8]:
# 학습모델을 실행
def train(total_loss):
    learning_rate = 0.0000001
    return tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)

In [9]:
# 학습모델을 검증
def evaluate(sess, X, Y):
    print ("[80., 25.]  to.." , sess.run(inference([[80., 25.]]))) # ~ 303
    print ("[65., 25.]  to.." , sess.run(inference([[65., 25.]]))) # ~ 256

In [10]:
# Launch the graph in a session, setup boilerplate
with tf.Session() as sess:
    training_steps = 1000   # 반복학습할 횟수를 지정 
    tf.global_variables_initializer().run()
    X, Y = inputs()   # X ,Y 값 입력
    total_loss = loss(X, Y)
    train_op = train(total_loss)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    for step in range(training_steps):
        sess.run([train_op])
        if step % 100 == 0:
            print (" {0:4}  loss: {1} ".format(step, sess.run([total_loss])))
    evaluate(sess, X, Y)
    coord.request_stop()
    coord.join(threads)
    sess.close()

    0  loss: [7608772.5] 
  100  loss: [5341925.5] 
  200  loss: [5339993.0] 
  300  loss: [5338747.0] 
  400  loss: [5337538.5] 
  500  loss: [5336334.0] 
  600  loss: [5335129.5] 
  700  loss: [5333925.5] 
  800  loss: [5332724.0] 
  900  loss: [5331523.0] 
[80., 25.]  to.. [[ 320.64968872]]
[65., 25.]  to.. [[ 267.78182983]]


## 4 Logistic Regression
로지스틱 회귀분석 : 데이터를 통해서 Yes - No 선택적 결론의 회귀식 계산

Titanic 데이터 분석 샘플 (with Python)

http://donnemartin.com/predicting-titanic-survivors.html

In [11]:
# Kaggle의 Titanic 데이터를 분석
# Download 1 https://www.kaggle.com/c/titanic/data
# Download 2 https://github.com/pcsanwald/kaggle-titanic/blob/master/train.csv
import tensorflow as tf; import os

# 선형회귀식의 가설함수를 재활용
W = tf.Variable(tf.zeros([5, 1]), name="weights")
b = tf.Variable(0., name="bias")

def combine_inputs(X):
    return tf.matmul(X, W) + b

In [12]:
# Sigmoid 를 사용
def inference(X):
    return tf.sigmoid(combine_inputs(X))

# Cross Entropy 를 사용
def loss(X, Y):
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=combine_inputs(X), labels=Y))

In [13]:
def read_csv(batch_size, file_name, record_defaults):
    filename_queue = tf.train.string_input_producer([os.path.join(os.getcwd(), file_name)])
    reader = tf.TextLineReader(skip_header_lines=1)
    key, value = reader.read(filename_queue)
    decoded = tf.decode_csv(value, record_defaults=record_defaults)
    return tf.train.shuffle_batch(decoded, batch_size=batch_size,
                                  capacity=batch_size * 50, min_after_dequeue=batch_size)

In [14]:
def inputs():
    survived, pclass, name, sex, age, sibsp, parch, ticket, fare, cabin, embarked = \
        read_csv(100, "./data/train.csv", [[0.0], [0], [""], [""], [0.0], [0.0], [0.0], [""], [0.0], [""], [""]])
    # convert categorical data
    is_first_class = tf.to_float(tf.equal(pclass, [1]))
    is_second_class = tf.to_float(tf.equal(pclass, [2]))
    is_third_class = tf.to_float(tf.equal(pclass, [3]))
    gender = tf.to_float(tf.equal(sex, ["female"]))
    # Finally we pack all the features in a single matrix;
    # We then transpose to have a matrix with one example per row and one feature per column.
    features = tf.transpose(tf.stack([is_first_class, is_second_class, is_third_class, gender, age]))
    survived = tf.reshape(survived, [100, 1])
    return features, survived

In [15]:
def train(total_loss):
    learning_rate = 0.01
    return tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)

In [16]:
def evaluate(sess, X, Y):
    predicted = tf.cast(inference(X) > 0.5, tf.float32)
    print (sess.run(tf.reduce_mean(tf.cast(tf.equal(predicted, Y), tf.float32))))

In [17]:
# Launch the graph in a session, setup boilerplate
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    X, Y = inputs()
    total_loss = loss(X, Y)
    train_op = train(total_loss)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    # actual training loop
    training_steps = 1000
    for step in range(training_steps):
        sess.run([train_op])
        # for debugging and learning purposes, see how the loss gets decremented thru training steps
        if step % 100 == 0:
            print (step,"loss: ", sess.run([total_loss]))
    evaluate(sess, X, Y)
    import time
    time.sleep(5)
    coord.request_stop()
    coord.join(threads)
    sess.close()

0 loss:  [0.94643354]
100 loss:  [0.69507045]
200 loss:  [0.61978805]
300 loss:  [0.62616438]
400 loss:  [0.78409117]
500 loss:  [0.57193363]
600 loss:  [0.57005286]
700 loss:  [0.54663479]
800 loss:  [0.53052485]
900 loss:  [0.5549221]
0.67


## 5 Softmax Classification
다중 선택형 모델 # Softma example in TF using the classical Iris dataset

출력 클래스에 n개의 뉴런을 사용한 신경망을 적용

In [18]:
# Download iris.data - https://archive.ics.uci.edu/ml/datasets/Iris
import tensorflow as tf; import os

# 4개의 특징값을 갖고, 3개의 출력 클래스를 갖는다 
# 때문에 weight matrix 는 4 X 3
W = tf.Variable(tf.zeros([4, 3]), name="weights")
b = tf.Variable(tf.zeros([3], name="bias")) # so do the biases, one per class.

def combine_inputs(X):
    return tf.matmul(X, W) + b

In [19]:
# tf.nn.softmax() : 하나의 클래스 값을 갖는 학습샘플의 최적화 softmax()
def inference(X):
    return tf.nn.softmax(combine_inputs(X))

# tf.nn.sparse_softmax_cross_entropy_with_logits() : A, B, C 중 하나의 클래스 값을 갖는 학습샘플의 최적화 softmax()
def loss(X, Y):
    return tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=combine_inputs(X), labels=Y))

In [20]:
def read_csv(batch_size, file_name, record_defaults):
    filename_queue = tf.train.string_input_producer([os.path.join(os.getcwd(), file_name)])
    reader = tf.TextLineReader(skip_header_lines=1)
    key, value = reader.read(filename_queue)
    decoded = tf.decode_csv(value, record_defaults=record_defaults)
    return tf.train.shuffle_batch(decoded, batch_size=batch_size,
                                  capacity=batch_size * 50, min_after_dequeue=batch_size)

In [21]:
# ./data/iris.data 의 Default 값을 호출
# tf.stack() 텐서를 생성
# tf.equal() 파일입력과 가능한 값을 비교
def inputs():
    sepal_length, sepal_width, petal_length, petal_width, label =\
        read_csv(100, "./data/iris.data", [[0.0], [0.0], [0.0], [0.0], [""]])
    # convert class names to a 0 based class index.
    label_number = tf.to_int32(tf.argmax(tf.to_int32(tf.stack([
                                tf.equal(label, ["Iris-setosa"]),
                                tf.equal(label, ["Iris-versicolor"]),
                                tf.equal(label, ["Iris-virginica"])])), 0))
    features = tf.transpose(tf.stack([sepal_length, sepal_width, petal_length, petal_width]))
    return features, label_number

In [22]:
# 학습 및 평가함수 : 
# tf.argmax() 참인 값을 갖는 텐서의 index 출력
def train(total_loss):
    learning_rate = 0.01
    return tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)

# 평가 결과는 tf.reduce_mean()를 적용
def evaluate(sess, X, Y):
    predicted = tf.cast(tf.arg_max(inference(X), 1), tf.int32)
    print(sess.run(tf.reduce_mean(tf.cast(tf.equal(predicted, Y), tf.float32))))

In [23]:
# 작성한 Graph 를 실행
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    X, Y = inputs()
    total_loss = loss(X, Y)
    train_op = train(total_loss)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    # actual training loop
    training_steps = 1000
    for step in range(training_steps):
        sess.run([train_op])
        # for debugging and learning purposes, see how the loss gets decremented thru training steps
        if step % 100 == 0:
            print (step, "loss: ", sess.run([total_loss]))

    evaluate(sess, X, Y)
    coord.request_stop()
    coord.join(threads)
    sess.close()

INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.InvalidArgumentError'>, Expect 5 fields but have 0 in record 0
	 [[Node: DecodeCSV_1 = DecodeCSV[OUT_TYPE=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_STRING], field_delim=",", _device="/job:localhost/replica:0/task:0/cpu:0"](ReaderReadV2_1:1, DecodeCSV_1/record_defaults_0, DecodeCSV_1/record_defaults_1, DecodeCSV_1/record_defaults_2, DecodeCSV_1/record_defaults_3, DecodeCSV_1/record_defaults_4)]]


OutOfRangeError: RandomShuffleQueue '_6_shuffle_batch_1/random_shuffle_queue' is closed and has insufficient elements (requested 100, current size 49)
	 [[Node: shuffle_batch_1 = QueueDequeueManyV2[component_types=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_STRING], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/cpu:0"](shuffle_batch_1/random_shuffle_queue, shuffle_batch_1/n)]]

Caused by op 'shuffle_batch_1', defined at:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/markbaum/Python/python-cpu/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/markbaum/Python/python-cpu/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/markbaum/Python/python-cpu/lib/python3.6/site-packages/ipykernel/kernelapp

## 6 Multi-Layer Nural Networks
여러 선형신경망의 가중치 합을 계산하며, 활성화(Activation) 및 전송(Transfer)함수를 적용하며

활성화 함수는 1)선형회귀는 항등함수를 2)로지스틱 회귀는 Sigmoid를 사용한다

cf)'선형분리'로 XOR는 부적합, 신경망 입출력에 '더 많은 뉴런'을 삽입함으로 써 극복 (Deep Learning 개념의 시작)

## 7 Gradient Descent & Backpropagation
tf.summary.scalar()   #손실을 텐서보드로 시각화 하는 함수